## Section 1: **l-alanine** geometry optimization

In the following cells, we will learn how to download a geometry of a molecule (in this case l-alanine) from pubchem, visualise its geometry, and check its rotational constants. Then, we will optimize its structure to a minimum
with quantum chemistry, and learn how to save its absolute energy, rotational constants, and dipole moments.

### Step 1.0: import python modules

In [ ]:
import psi4                        # for quantum chemistry
import py3Dmol as p3d              # for 3D visualisation
import qcelemental as qcel         # for physical constants
from rdkit import Chem             # for 2D visualisation
from openbabel import openbabel    # for format conversions
from numpy import linalg           # for linear algebra

### Step 1.1: getting l-alanine from pubchem:

This is fairly easy. We tell Psi4 to search for a geometry of `<molecule>` on pubchem using the `psi4.geometry("pubchem:<molecule>")` syntax. This function returns a special `molecule` object, which contains the geometry of the molecule (atomic positions in cartesian coordinates), information about the molecular charge, multiplicity, and derived data such as rotational constants and the distance matrix between all atoms. As we want to use the geometry later, we save it into a variable called `l_alanine`:

In [ ]:
l_alanine = psi4.geometry("pubchem:l-alanine")

### Step 1.2: visualising the molecule:
#### Step 1.2.1: 3D representation:
Here we define a function `drawMol3D()` that takes one argument, a molecule geometry in the Psi4 format. You can adjust the width and height of the generated figure by adjusting the parameters. The function first generates a `XYZ file` representation of the molecule, and draws it using py3Dmol.

We can re-use this function by calling `drawMol3D(<saved molecule>)`, or in the case of l-alanine, `drawMol3D(l_alanine)`. In the example below, we save this "view" into the variable `fig`, so that we can generate a `png` figure for our lab report. You can rotate the figure by dragging the mouse, and zoom by using the scroll wheel.

In [ ]:
def drawMol3D(mol, width = 250, height = 200):
    xyz = mol.save_string_xyz_file() 
    view = p3d.view(width = width, height = height)
    view.addModel(xyz, "xyz")
    view.setStyle({'stick':{}})
    view.zoomTo()
    return view
fig = drawMol3D(l_alanine)
fig.show()

Once you are happy with how the molecule looks in the box above, run the cell below, and then you can `right click -> save as` the figure.

In [ ]:
fig.png()

#### Step 1.2.2: 2D stereochemical representation / diagram:
For this visualisation, we use two very powerful packages: OpenBabel and RDkit. First, we need to convert the Psi4 molecule into a SMILES representation. We can generate the XYZ coordinates directly from the Psi4 format. Then we need to convert that to SMILES using openbabel. Then, we can use these SMILES to generate a drawing that displays stereochemistry using RDkit. 

In [ ]:
def drawMol2D(mol, width=100, height=100):
    xyz = mol.save_string_xyz_file()
    obc = openbabel.OBConversion()
    obm = openbabel.OBMol()
    obc.SetInAndOutFormats("xyz", "smi")
    obc.ReadString(obm, xyz)
    smi = obc.WriteString(obm)
    print("The generated SMILES are: " + smi)
    m = Chem.MolFromSmiles(smi)
    Chem.Kekulize(m)
    return(m)

drawMol2D(l_alanine)

A further example with a few randomly chosen molecules is shown below.

In [ ]:
names = ["l-glucose", "butadiene", "nicotine", "l-tyrosine", "d-alanine", "l-alanine"]
pics = []
for name in names:
    mol = psi4.geometry(f"pubchem:{name}")
    pics.append(drawMol2D(mol))
Chem.Draw.MolsToGridImage(pics)

### Step 1.3: Accessing rotational constants
We can also check the rotational constants of the original molecule from pubchem. The function `<molecule>.rotational_constants().np` returns the rotational constants as a wavenumber ($\lambda$) in 
cm<sup>-1</sup>. For the rest of this module, we'd prefer to use rotational constants as frequencies ($f$) 
in MHz, therefore we need to convert between the two using:

\begin{align}
f = c \lambda
\end{align}

We can define another helper function to aid us. The module `qcelemental.constants` contains all important physical constants, and we can access the speed of light ($c$, in m/s) as `qcel.constants.c`. We need to multiply that by 100 (to convert from m/s to cm/s), and then convert from Hz (which is s <sup>-1</sup>) to MHz.

In [ ]:
def getRotConstMHz(mol):
    incm = mol.rotational_constants().np
    inHz = [i * qcel.constants.c * 100 for i in incm]
    inMHz = [i/1e6 for i in inHz]
    return(inMHz)

pubchem_rot_consts = getRotConstMHz(l_alanine)
print(pubchem_rot_consts)

We unfortunately cannot check the dipole moment of the pubchem version, as for that we need the electron density. We will look at that using the optimized version of the molecule.

### Step 1.4: Optimizing the geometry with Psi4:
When we want to find the minimum energy conformation of any molecule, the process by
which we do that is called "geometry optimization". We will use the program Psi4 to do that, 
using a good but computationally cheap method: [PBEh-3c by Grimme et al.](https://doi.org/10.1063/1.4927476). This method is quite nice, as it includes modern corrections, but remains computationally inexpensive.

The pubchem geometry of l-alanine is close to the true minimum geometry, but each computational method is slightly different, so it makes sense to make sure we are at the minimum of that method. This is especially important as we are after the rotational constants.

In the next cell, we will optimize the l-alanine molecule stored in `l_alanine` using the PBEh-3c density functional. **This may take a few minutes, so do not panic.** The absolute energy of the optimized molecule will be saved into the variable `E` (in Hartrees), and the wavefunction $\psi$ of the optimized molecule into the `wfn` object. The new geometry can be accessed as `wfn.molecule()`.

Once finished, a message "Optimizer: Optimization complete!" will appear...

In [ ]:
E, wfn = psi4.optimize("pbeh3c", molecule=l_alanine, return_wfn=True)

### Step 1.5: Compare pubchem and optimized l-anilines:

In the cell below, we compare the original (from pubchem) and optimized 3D representations of l-aniline. Is there any perceptible difference?

In [ ]:
print("pubchem version:")
drawMol3D(l_alanine).show()
print(getRotConstMHz(l_alanine))
print()
print("optimized version:")
drawMol3D(wfn.molecule()).show()
print(getRotConstMHz(wfn.molecule()))

As you see in the output above, there is a significant change in the rotational constants, especially along $b$ and $c$, even though visually the change is almost imperceptible.

### Step 1.6: Accessing dipole moments:
The dipole moments are stored in the wavefunction object `wfn`. We can access them using the `wfn.array_variable("CURRENT DIPOLE").np` function as shown below:

In [ ]:
print(wfn.array_variable("CURRENT DIPOLE").np)

Unfortunately, there are two issues with this dipole moment vector:

- the dipole moments are in atomic units ($e a_0$) as opposed to Debye
- the orientation is in Cartesian (XYZ) coordinates as opposed to the principal axis system (ABC)

To convert the units, we can simply use the conversion factor `qcel.constants.dipmom_au2debye`. 

In [ ]:
dipole_XYZ = wfn.array_variable("CURRENT DIPOLE").np * qcel.constants.dipmom_au2debye
print(dipole_XYZ)

For the second problem, we need to figure out how to rotate our molecule so that the principal axes (ABC) are parallel with Cartesian axes (XYZ). We can do this by solving the eigenvalue equation of the moment of inertia tensor, ordering the eigenvalues and eigenvectors so that the eigenvalues, which are the principal moments of inertia $I_a, I_b, I_c$, are sorted from smallest to largest, and then applying the ordered eigenvectors to rotate the dipole moment vector from XYZ to ABC frame:

In [ ]:
I = wfn.molecule().inertia_tensor().np
evals, evecs = linalg.eig(I)
idx = evals.argsort()
evals = evals[idx]
evecs = evecs[:,idx]
dipole_ABC = dipole_XYZ.dot(evecs)
print(dipole_ABC)

To illustrate this rotation, we can show the 3D structures before and after applying the transformation:

In [ ]:
print("Original molecule:")
display(drawMol3D(wfn.molecule()))
print(dipole_XYZ)
print()
print("Rotated molecule:")
rotmol = wfn.molecule().clone()
mat = psi4.core.Matrix(*rotmol.geometry().shape).from_array(rotmol.geometry().np.dot(evecs))
rotmol.set_geometry(mat)
display(drawMol3D(rotmol))
print(dipole_ABC)

To make the process of getting the rotated dipole moment components in correct units a little easier, we have provided you with the following function:

In [ ]:
def getDipoleABC(wf):
    dipole_XYZ = wf.array_variable("CURRENT DIPOLE").np * qcel.constants.dipmom_au2debye
    I = wf.molecule().inertia_tensor().np
    evals, evecs = linalg.eig(I)
    idx = evals.argsort()
    evals = evals[idx]
    evecs = evecs[:,idx]
    dipole_ABC = dipole_XYZ.dot(evecs)
    return(dipole_ABC)

getDipoleABC(wfn)